In [5]:
pip install numpy

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/12.6 MB 9.4 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/12.6 MB 7.8 MB/s eta 0:00:02
   ------------- -------------------------- 4.2/12.6 MB 6.9 MB/s eta 0:00:02
   ---------------- ----------------------- 5.2/12.6 MB 6.7 MB/s eta 0:00:02
   ------------------- -------------------- 6.3/12.6 MB 6.2 MB/s eta 0:00:02
   ---------------------- ----------------- 7.1/12.6 MB 5.7 MB/s eta 0:00:01
   ------------------------ --------------- 7.9/12.6 MB 5.5 MB/s eta 0:00:01
   --------------------------- ------------ 8.7/12.6 MB 5.3 MB/s eta 0:00:01
   ----------------------------- ---------- 9.4/12.6 MB 5.1 MB/s eta 0:00:01
   -------------------------------- ------- 10.2/12.6 MB 5.0 MB/s eta 0:00:01
   ---------------------------------- ----- 10.7/12.6 MB 4.8 MB/s eta 0:00:01
   ---------------------------------- ----- 11.0/12.6 MB 4.6 MB/s eta 0:00:01
   

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import json
import time
# seeds = np.random.randint(1,1000, size = 5)
seeds = [153, 442, 929, 384, 324]

# GA (Genetic Algorithm)

In [7]:
def randomSolution(bases_count, limit):
  solution = np.zeros(bases_count, dtype=bool)
  indexes = rng.choice(list(range(bases_count)), limit, replace= False)
  solution[indexes] = 1
  return solution

In [8]:
def mutation(solution):
  ones = np.where(solution == 1)[0]
  zeros = np.where(solution == 0)[0]
  one = rng.choice(ones, 1)
  zero = rng.choice(zeros, 1)
  solution = np.copy(solution)
  solution[one] = 0
  solution[zero] = 1
  return solution

In [9]:
def crossover(solution1,solution2):
  child1 = solution1*solution2
  child2 = solution1*solution2

  uniques_solution = solution1 + solution2 - child1*2

  ones = np.where(uniques_solution == 1)[0]
  rng.shuffle(ones)
  halfones = len(ones)//2
  firsthalf = ones[:halfones]
  secondhalf = ones[halfones:]

  child1[firsthalf] = 1
  child2[secondhalf] = 1

  return (child1,child2)

In [10]:
def tournament_selection(population_fitness, tournamentk, all_pop_ind, pop_size):
  selection = []
  for i in range(pop_size-2):
    individuals = rng.choice(all_pop_ind, size=tournamentk, replace=False)
    bestFitness = 0
    best = -1
    for i in range(tournamentk):
      this_fitness = population_fitness[individuals[i]]
      if(this_fitness > bestFitness):
        bestFitness = this_fitness
        best = individuals[i]
    selection.append(best)
  return selection

In [11]:
def reproduction(population, pop_size, selection, pmut):
  newPopulation = []
  for i in range(pop_size//2-1):
    ind1 = population[selection[i*2]]
    ind2 = population[selection[i*2+1]]
    typeReproduction = rng.random()
    if typeReproduction < pmut:
      # mutacao
      newPopulation.append(mutation(ind1))
      newPopulation.append(mutation(ind2))
    else:
      # crossover
      (new1,new2) = crossover(ind1, ind2)
      newPopulation.append(new1)
      newPopulation.append(new2)
  return newPopulation

In [12]:
def objectiveFunction(solution, coverMatrix, numPeopleHelpedList):
  coverage = coverMatrix[:,solution]
  coverage = np.logical_or.reduce(coverage, axis=1)
  solutionUtility = coverage*numPeopleHelpedList
  return np.sum(solutionUtility)

In [13]:
def solveGeneticAlgorithm(pop_size, ngen, pmut, tournamentk, coverMatrix, numPeopleHelpedList, limit):
  allPop = list(range(pop_size))
  bases_count = coverMatrix.shape[1]

  population = [randomSolution(bases_count, limit) for i in range(pop_size)]
  for i in range(ngen):
    population_fitness = [objectiveFunction(ind, coverMatrix, numPeopleHelpedList) for ind in population]
    # if i % (ngen//4) == 0:
      # print(i, np.mean(population_fitness), np.max(population_fitness))
    indexBest = population_fitness.index(np.max(population_fitness))
    best = population[indexBest].copy()

    selection = tournament_selection(population_fitness, tournamentk, allPop, pop_size)
    newPopulation = reproduction(population, pop_size, selection, pmut)

    # elitism
    newPopulation.append(best)
    newPopulation.append(best)

    population = newPopulation

  population_fitness = [objectiveFunction(ind, coverMatrix, numPeopleHelpedList) for ind in population]
  print(i, np.mean(population_fitness), np.max(population_fitness))

  indexBest = population_fitness.index(max(population_fitness))
  best = population[indexBest].copy()
  return best

In [15]:
resultDict = {}
f = open('/content/casos_teste2x.json')
data = json.load(f)
for instance in data.keys():
  data[instance]['matriz'] = np.asarray(data[instance]['matriz'])
  data[instance]['populacao'] = np.asarray(data[instance]['populacao'])

FileNotFoundError: [Errno 2] No such file or directory: '/content/casos_teste2x.json'

In [14]:
data.keys()

NameError: name 'data' is not defined

In [ ]:
matriz = data['kroA150']['matriz']
populacao = data['kroA150']['populacao']
p = int(np.ceil(matriz.shape[1]*0.2))



result = []
for seed in seeds:
  rng = np.random.default_rng(seed=seed)
  best = solveGeneticAlgorithm(400, 100, 0.7, 5, coverMatrix = matriz, numPeopleHelpedList = populacao, limit = p)
  result.append(objectiveFunction(best,matriz,populacao))
result = np.array(result)

99 48715.8075 52304
99 48865.1775 52304


KeyboardInterrupt: ignored

In [ ]:
resultDict = {}

In [ ]:
for instance in data.keys():
  print(instance)
  matriz = data[instance]['matriz']
  populacao = data[instance]['populacao']
  p = int(np.ceil(matriz.shape[1]*0.2))

  result = []
  duration = []
  for seed in seeds:
    rng = np.random.default_rng(seed=seed)
    start_time = time.time()
    best = solveGeneticAlgorithm(400, 100, 0.8, 5, coverMatrix = matriz, numPeopleHelpedList = populacao, limit = p)
    end_time = time.time()
    duration.append(end_time-start_time)

    result.append(objectiveFunction(best,matriz,populacao))
  result = np.array(result)
  duration = np.array(duration)
  resultDict[instance] = {'mean': result.mean(), 'std': result.std(), 'duration':duration.mean()}

att48
99 18164.6025 21097
99 18121.5275 21097
99 18381.0675 21097
99 17982.1025 21097
99 18071.945 21097
berlin52
99 20175.225 22414
99 19708.555 22414
99 20209.9575 22414
99 19995.95 22414
99 20113.39 22414
kroA100
99 24247.5125 28066
99 23985.5575 28066
99 24168.495 28066
99 24387.6875 28066
99 24152.58 28066
kroA150
99 47262.13 52304
99 47814.92 52304
99 47008.73 52304
99 47491.1275 52304
99 47818.1925 52304
kroA200
99 66756.2475 72776
99 66517.0825 72776
99 66943.2525 72776
99 67183.52 72776
99 66896.82 72776
kroB100
99 33788.61 38819
99 33658.0375 38819
99 33791.88 38819
99 33865.12 38819
99 33765.865 38819
kroB150
99 51433.375 56654
99 51368.875 56654
99 51415.22 56654
99 51669.4525 56654
99 51639.92 56654
kroB200
99 60174.5675 65297
99 60320.26 65297
99 60501.7575 65297
99 59826.2175 65297
99 60322.815 65297
kroC100
99 22875.015 25778
99 22657.7775 25778
99 22996.615 25778
99 22707.035 25778
99 22901.9175 25778
kroD100
99 30042.675 35413
99 30524.06 35413
99 30848.685 35413
99 2

In [ ]:
resultDict

{'att48': {'mean': 21097.0, 'std': 0.0, 'duration': 5.0612263679504395},
 'berlin52': {'mean': 22414.0, 'std': 0.0, 'duration': 5.09383454322815},
 'kroA100': {'mean': 28066.0, 'std': 0.0, 'duration': 4.859729814529419},
 'kroA150': {'mean': 52304.0, 'std': 0.0, 'duration': 5.085614538192749},
 'kroA200': {'mean': 72776.0, 'std': 0.0, 'duration': 5.244046545028686},
 'kroB100': {'mean': 38819.0, 'std': 0.0, 'duration': 5.103917026519776},
 'kroB150': {'mean': 56654.0, 'std': 0.0, 'duration': 5.18713755607605},
 'kroB200': {'mean': 65297.0, 'std': 0.0, 'duration': 4.88747124671936},
 'kroC100': {'mean': 25778.0, 'std': 0.0, 'duration': 5.130774784088135},
 'kroD100': {'mean': 35413.0, 'std': 0.0, 'duration': 5.098598766326904},
 'kroE100': {'mean': 36824.0, 'std': 0.0, 'duration': 5.162283802032471},
 'lin105': {'mean': 32918.0, 'std': 0.0, 'duration': 4.923824977874756},
 'pr107': {'mean': 15978.0, 'std': 0.0, 'duration': 5.000982475280762},
 'pr124': {'mean': 48556.0, 'std': 0.0, 'dur

In [ ]:
for result in resultDict:
  print(resultDict[result]['duration'])

5.0612263679504395
5.09383454322815
4.859729814529419
5.085614538192749
5.244046545028686
5.103917026519776
5.18713755607605
4.88747124671936
5.130774784088135
5.098598766326904
5.162283802032471
4.923824977874756
5.000982475280762
5.176212024688721
5.180799388885498
5.18347749710083
4.80505337715149
5.101042747497559
5.2375
5.171657276153565
4.849347305297852


In [ ]:
with open('resultGA2.json', 'w') as json_file:
    json.dump(resultDict, json_file)

In [ ]:
for i in resultDict:
  print(i, ',', resultDict[i]['duration'],';')

att48 , 5.0612263679504395 ;
berlin52 , 5.09383454322815 ;
kroA100 , 4.859729814529419 ;
kroA150 , 5.085614538192749 ;
kroA200 , 5.244046545028686 ;
kroB100 , 5.103917026519776 ;
kroB150 , 5.18713755607605 ;
kroB200 , 4.88747124671936 ;
kroC100 , 5.130774784088135 ;
kroD100 , 5.098598766326904 ;
kroE100 , 5.162283802032471 ;
lin105 , 4.923824977874756 ;
pr107 , 5.000982475280762 ;
pr124 , 5.176212024688721 ;
pr136 , 5.180799388885498 ;
pr144 , 5.18347749710083 ;
pr152 , 4.80505337715149 ;
pr76 , 5.101042747497559 ;
rat195 , 5.2375 ;
rat99 , 5.171657276153565 ;
st70 , 4.849347305297852 ;


In [ ]:
resultDict